# Amortizirana časovna zahtevnost

Operacija v podatkovni strukturi ima _amortizirano časovno zahtevnost_ $C$, če vsako zaporedje $n$ takih operacij porabi največ $C ⋅ n$ enot časa.

## Primeri

### Dvojiški števec

Števec predstavimo s tabelo $B$, ki predstavlja število $\sum_{i = 0}^\infty B[i] \cdot 2^i$.

In [ ]:
VELIKOST = 4
KORAKI = 10

def izpisi(stevec):
    print("".join(str(b) for b in reversed(stevec)))

stevec = [0] * VELIKOST
for korak in range(KORAKI):
    izpisi(stevec)
    i = 0
    while stevec[i] == 1:
        stevec[i] = 0
        i += 1
    stevec[i] = 1
izpisi(stevec)


In [ ]:
import plotly.graph_objects as go
from collections import defaultdict

class CostTracker:
    def __init__(self, title):
        self.title = title
        self.costs = defaultdict(int)
    
    def add_cost(self, step, cost):
        self.costs[step] += cost
    
    def plot(self):
        steps = sorted(self.costs.keys())
        costs = [self.costs[s] for s in steps]
        
        if len(steps) <= 128:
            fig = go.Figure(data=go.Bar(x=steps, y=costs))
        else:
            fig = go.Figure(data=go.Scatter(x=steps, y=costs, mode="lines"))
        fig.update_layout(
            xaxis_title="Korak",
            yaxis_title=self.title
        ).show()

In [ ]:
VELIKOST = 8
KORAKI = 100

spremembe_bitov = CostTracker("Spremenjeni biti")

stevec = [0] * VELIKOST
for korak in range(KORAKI):
    i = 0
    while stevec[i] == 1:
        stevec[i] = 0
        spremembe_bitov.add_cost(step=korak, cost=1)
        i += 1
    stevec[i] = 1
    spremembe_bitov.add_cost(step=korak, cost=1)

spremembe_bitov.plot()

Vsak `increment` porabi $O(\log n)$ časa. Torej $n$ operacij porabi $O(n \log n)$ časa. Znamo bolje?

### Dinamična tabela

#### Lastna implementacija

In [ ]:
class DynamicTable:
    def __init__(self, cost_tracker):
        self._size = 0
        self._data = [None]
        self._cost_tracker = cost_tracker

    @property
    def _capacity(self):
        return len(self._data)

    def __len__(self):
        return self._size

    def __getitem__(self, index):
        if index < 0:
            index += self._size
        return self._data[index]

    def append(self, value):
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        self._data[self._size] = value
        self._cost_tracker.add_cost(step=self._size, cost=1)
        self._size += 1

    def _resize(self, new_capacity):
        new_data = [None] * new_capacity
        for i in range(self._size):
            new_data[i] = self._data[i]
        self._cost_tracker.add_cost(step=self._size, cost=self._size)
        self._data = new_data

In [ ]:
VELIKOST_TABELE = 10_000

stevilo_vstavljenih = CostTracker("Vstavljeni elementi")
tabela = DynamicTable(stevilo_vstavljenih)

for korak in range(1, VELIKOST_TABELE):
    tabela.append(korak)

stevilo_vstavljenih.plot()

Vsak `append` porabi $O(n)$ časa. Torej $n$ operacij porabi $O(n^2)$ časa. Znamo bolje?

#### Vgrajena implementacija

In [ ]:
import sys
import time

STEVILO_POSKUSOV = 1
VELIKOST_TABELE = 1_000

porabljen_cas = CostTracker()

for _ in range(STEVILO_POSKUSOV):
    tabela = []
    for korak in range(VELIKOST_TABELE):
        kapaciteta = sys.getsizeof(tabela)
        zacetek = time.time()
        tabela.append(korak)
        konec = time.time()
        nova_kapaciteta = sys.getsizeof(tabela)
        if nova_kapaciteta > kapaciteta:
            print(f"{korak}: {kapaciteta} → {nova_kapaciteta} - {nova_kapaciteta / kapaciteta:.3f}×")
        porabljen_cas.add_cost(step=korak, cost=konec - zacetek)

porabljen_cas.plot()

## Agregacijska metoda

Pri agregacijski metodi amortizirano časovno zahtevnost izračunamo tako, da seštejemo čase vseh operacij in izračunamo povprečje:

$$C = \frac{\sum_{i = 1}^n C_i} n ⟹ \sum_{i = 1}^n C_i ≤ C ⋅ n$$

### Dvojiški števec

V $n$ operacijah `increment` se:
- bit $B[0]$ spremeni na vsakem koraku, torej $n$-krat;
- bit $B[1]$ spremeni na vsakem drugem koraku, torej $⌊n / 2⌋$-krat;
- bit $B[2]$ spremeni na vsakem četrtem koraku, torej $⌊n / 4⌋$-krat;
- ⋮
- bit $B[i]$ spremeni na vsakem $2^i$-koraku, torej $⌊n / 2^{\log n}⌋$-krat

Torej je skupno število sprememb v $n$ korakih:
$$
    n + \left⌊\frac{n}{2}\right⌋ + \left⌊\frac{n}{4}\right⌋ + ⋯ + \left⌊\frac{n}{2^{\log n}}\right⌋
    =
    \sum_{i = 0}^∞ \left⌊\frac{n}{2^i}\right⌋
    ≤
    \sum_{i = 0}^∞ \frac{n}{2^i}
    =
    2 n
    
$$

Skupna časovna zahtevnost $n$ operacij je $O(n)$, torej je amortizirana časovna zahtevnost $O(1)$.

### Dinamična tabela

V $n$ operacijah `append` se:
- na vsakem koraku vstavi en element, torej $n$ vstavljanj;
- na koraku $2^i$ vstavimo še dodatnih $2^i$ elementov v novo tabelo.

Torej je skupno število vstavljanj v $n$ korakih:
$$
    n + 2^1 + 2^2 + 2^3 + ⋯ + 2^{⌊\log n⌋}
    =
    n + \sum_{i = 1}^{⌊\log n⌋} 2^i
    =
    n + 2^{⌊\log n⌋ + 1} - 1
    ≤
    n + 2^{\log n + 1}
    =
    3 n
$$

Skupna časovna zahtevnost $n$ operacij je $O(n)$, torej je amortizirana časovna zahtevnost $O(1)$.

## Računovodska metoda

Vsaka operacija lahko prevzame del stroška prihodnjih operacij. To storimo tako, da med izvajanjem beležimo porabo (virtualno, ne v dejanski implementaciji), operacije pa potem bodisi „plačajo davek“ za prihodnje operacije oziroma so plačana iz poprej vplačanih davkov.

### Dvojiški števec

Vsak bit dobi svojo „žepnino“, cene operacij pa so sledeče:

- Sprememba bita $0 → 1$ plača dve enoti, eno porabi za spremembo, drugo pa si shrani za naslednjo morebitno spremembo $1 → 0$.
- Sprememba bita $1 → 0$ ne plača ničesar, saj je bila sprememba že pokrita v spremembi $0 → 1$.

Na vsakem koraku lahko naredimo več sprememb $1 → 0$, ampak _samo eno_ spremembo $0 → 1$. Zato skupaj plačamo $2 n$ enot in amortizirana časovna zahtevnost je $O(1)$.

In [ ]:
EURO = "€"
NI_EURA = " "

VELIKOST = 4
KORAKI = 10

stevec = [0] * 8
zepnina = [NI_EURA] * 8
for korak in range(KORAKI):
    izpisi(zepnina)
    izpisi(stevec)
    print()
    i = 0
    while stevec[i] == 1:
        stevec[i] = 0
        assert zepnina[i] == EURO
        zepnina[i] = NI_EURA
        i += 1
    stevec[i] = 1
    zepnina[i] = EURO
izpisi(zepnina)
izpisi(stevec)


### Dinamična tabela

Vsako vstavljanje stane tri enote:

- Ena enota se porabi za vstavljanje elementa v tabelo.
- Druga enota se porabi za prvo kasnejše prestavljanje v večjo tabelo.
- Tretja enota se porabi za prestavljanje nekega že prestavljenega elementa v večjo tabelo.

### Disjunktne množice

In [ ]:
class DisjointSet:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [0] * size

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        px = self.find(x)
        py = self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1
        return True

Dokažimo, da $m$ klicev metode `find` na $n$ elementih porabi $O(m \log^* n) + O(n \log^* n)$ časa. Pri tem je običajno $n ≤ m$, torej je amortizirana časovna zahtevnost $O(\log^* n)$.

Naj bo $I_i = \{ x ∈ \mathbb{N} | \log^* x = i \}$:

$$
I_0 = [1, 1] \qquad
I_1 = [2, 2] \qquad
I_2 = [3, 4] \qquad
I_3 = [5, 16] \qquad
I_4 = [17, 2^{16}] \qquad
I_5 = [2^{16} + 1, 2^{2^{16}}]
$$

Ko vozlišče ranga $r ∈ [ℓ + 1, 2^ℓ]$ preneha biti koren, mu damo $2^ℓ$ žepnine. Vsak koren ranga $k$ ima vsaj $2^k$ potomcev. Zato mora biti vozlišč ranga $k$ kvečjemu $n / 2^k$, torej jih je na tem intervalu kvečjemu

$$
\frac{n}{2^{ℓ + 1}} + \frac{n}{2^{ℓ + 2}} + ⋯ + \frac{n}{2^{2^ℓ}} < \frac{n}{2^ℓ}
$$

zato za njih porabimo največ $n$ žepnine. Ker je intervalov $\log^* n$, smo podelili $O(n \log^* n)$ žepnine.

Vsaka operacija `find` porabi toliko časa, kot je dolga njena pot $x = y_0 → y_1 → ⋯ → y_r$. Vrednosti $\mathop{\mathrm{rang}(y_i)}$ in $\mathop{\mathrm{rang}(y_{i + 1})}$ sta lahko:
- na različnih intervalih (kar se lahko zgodi največ $(\log^* n)$-krat)
- na istem intervalu $[ℓ + 1, 2^ℓ]$, pri čemer bomo strošek plačali iz žepnine $y_i$. Ker bomo $y_i$ prevezali največ $2^ℓ$-krat, ima dovolj žepnine.

## Potencialna metoda

## Semidinamična tabela